In [4]:
import re
import os
import collections
import time

In [5]:
# This is the map where dictionary terms will be stored as keys and value will be posting list with position in the file
dictionary = {}
matrix = {}
# This is the map of docId to input file name
docIdMap = {}

class index:
    def __init__(self, path):
        self.path = path
        pass

    def buildIndex(self):

        docId = 1
        fileList = [f for f in os.listdir(self.path) if os.path.isfile(os.path.join(self.path, f))]
        fileobj = open('frequency.txt', 'w')
        fileobj1 = open('matrix.txt', 'w')
        for eachFile in fileList:
            position = 1
            count = 0
            # docName = "Doc_Id_" + str(docId)
            # docName =  str(docId)
            docIdMap[docId] = eachFile
            lines = [line.rstrip('\n') for line in open(self.path + "/" + eachFile)]

            for eachLine in lines:
                wordList = re.split('\W+', eachLine)

                while '' in wordList:
                    wordList.remove('')
                for word in wordList:
                    if (word.lower() in dictionary):
                        postingList = dictionary[word.lower()]
                        if (docId in postingList):
                            postingList[docId].append(position)
                            position = position + 1
                        else:
                            postingList[docId] = [position]
                            position = position + 1
                    else:
                        dictionary[word.lower()] = {docId: [position]}
                        position = position + 1
            docId = docId + 1
        #lengths = {key:len(value) for key,value in dictionary.iteritems()}
        length_dict = {key: len(value) for key, value in dictionary.items()}
        for w in length_dict:
            fileobj.write(w +"   |   "+str(length_dict[w]))
            fileobj.write("\n")
        fileobj.close()


        firstLine = "          **|**   "
        for d in docIdMap:
            firstLine = firstLine + " | " +str(d)
        fileobj1.write(firstLine+"   **|** ")
        fileobj1.write('\n')
        fileobj1.write('\n')
        for t in dictionary:
            poList = dictionary[t]
            kList = []
            for keys in poList:
                kList.append(keys)
            line = "       "

            for d in docIdMap:
                if d in kList:
                    line = line + " | " + "1"
                else:
                    line = line + " | " + "0"

            fileobj1.write(t+"  **|**"+line)
            fileobj1.write('\n')

        fileobj1.close()

    def and_query(self, query_terms):
        if len(query_terms) == 1:
            resultList = self.getPostingList(query_terms[0])
            if not resultList:
                print ("")
                printString = "Result for the Query : " + query_terms[0]
                print (printString)
                print ("0 documents returned as there is no match")
                return

            else:
                print ("")
                printString = "Result for the Query : " + query_terms[0]
                print (printString)
                print ("Total documents retrieved : " + str(len(resultList)))
                for items in resultList:
                    print(docIdMap[items])

        else:
            resultList = []
            for i in range(1, len(query_terms)):
                if (len(resultList) == 0):
                    resultList = self.mergePostingList(self.getPostingList(query_terms[0]),
                                                       self.getPostingList(query_terms[i]))
                else:
                    resultList = self.mergePostingList(resultList, self.getPostingList(query_terms[i]))
            print ("")
            printString = "Result for the Query(AND query) :"
            i = 1
            for keys in query_terms:
                if (i == len(query_terms)):
                    printString += " " + str(keys)
                else:
                    printString += " " + str(keys) + " AND"
                    i = i + 1

            print (printString)
            print ("Total documents retrieved : " + str(len(resultList)))
            for items in resultList:
                print (docIdMap[items])

    def getPostingList(self, term):
        if (term in dictionary):
            postingList = dictionary[term]
            keysList = []
            for keys in postingList:
                keysList.append(keys)
            keysList.sort()
            # print keysList
            return keysList
        else:
            return None

    def mergePostingList(self, list1, list2):

        mergeResult = list(set(list1) & set(list2))
        mergeResult.sort()
        return mergeResult

    def print_dict(self):
        # function to print the terms and posting list in the index
        fileobj = open("invertedIndex.txt", 'w')
        for key in dictionary:
            print (key + " --> " + str(dictionary[key]))
            fileobj.write(key + " --> " + str(dictionary[key]))
            fileobj.write("\n")
        fileobj.close()

    def print_doc_list(self):
        for key in docIdMap:
            print ("Doc ID: " + str(key) + " ==> " + str(docIdMap[key]))


def main():
    docCollectionPath = input("Enter path of text file collection : ")
    queryFile = input("Enter path of query file : ")
    indexObject = index(docCollectionPath)
    indexObject.buildIndex()

    print ("")
    print ("Inverted Index :")
    indexObject.print_dict()

    print ("")
    print ("Document List :")
    indexObject.print_doc_list()
    print ("")

    QueryLines = [line.rstrip('\n') for line in open(queryFile)]
    for eachLine in QueryLines:
        wordList = re.split('\W+', eachLine)

        while '' in wordList:
            wordList.remove('')

        wordsInLowerCase = []
        for word in wordList:
            wordsInLowerCase.append(word.lower())
        indexObject.and_query(wordsInLowerCase)

if __name__ == '__main__':
    main()

Enter path of text file collection : C:\Users\maede\Documents\Unstructured data\Assignment2\Document
Enter path of query file : C:\Users\maede\Documents\Unstructured data\Assignment2

Inverted Index :
3d --> {1: [1, 13, 81]}
radiology --> {1: [2, 21, 194]}
lab --> {1: [3, 182, 254, 265]}
stanford --> {1: [4, 9, 26, 141, 155, 170, 188, 266, 270, 283, 323, 356, 366], 2: [1, 6, 11, 15, 24, 70, 89, 104, 375, 396, 398, 410, 418, 478, 481, 494, 534, 567, 574], 3: [75, 90]}
university --> {1: [5, 357], 2: [7, 90, 568]}
school --> {1: [6, 10, 190, 294, 309, 367], 2: [8, 12, 91, 100, 420, 505, 520, 575]}
of --> {1: [7, 11, 20, 59, 97, 123, 135, 191, 252, 295, 310, 368, 371], 2: [9, 13, 74, 81, 92, 97, 128, 131, 139, 146, 152, 158, 176, 208, 212, 339, 421, 506, 521, 576, 579], 3: [28, 58, 72, 82, 120, 139, 160, 170, 185, 220, 232, 246, 267, 273, 285]}
medicine --> {1: [8, 12, 189, 192, 267, 271, 296, 311, 369], 2: [10, 14, 93, 419, 422, 479, 482, 507, 522, 577]}
and --> {1: [14, 49, 57, 71, 74, 

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\maede\\Documents\\Unstructured data\\Assignment2'